## Лабораторная работа №1 (курс "Математические методы анализа текстов")

#### Тема: Определение частей речи и выделение именованных сущностей.


**Выдана**:   25 февраля 2017

**Дедлайн**:   <font color='red'>9:00 утра 13 марта 2017</font>

**Среда выполнения**: Jupyter Notebook (Python 2.7)

#### Правила:

Результат выполнения задания - отчет в формате Jupyter Notebook с кодом и выводами. В ходе выполнения задания требуется реализовать все необходимые алгоритмы, провести эксперименты и ответить на поставленные вопросы. Дополнительные выводы приветствуются. Чем меньше кода и больше комментариев - тем лучше.

Все ячейки должны быть "выполненными", при этом результат должен воспроизвдиться при проверке (на Python 2.7). Если какой-то код не был запущен или отрабатывает с ошибками, то пункт не засчитывается. Задание, сданное после дедлайна, _не принимается_. Совсем.


Задание выполняется самостоятельно. Вы можете обсуждать идеи, объяснять друг другу материал, но не можете обмениваться частями своего кода. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов, а также предвзято негативное отношение семинаристов в будущем. Если вы нашли в Интернете какой-то код, который собираетесь заимствовать, обязательно укажите это в задании: вполне вероятно, что вы не единственный, кто найдёт и использует эту информацию.

#### Постановка задачи:

В данной лабораторной работе вам предстоит:

- обучить скрытую марковскую модель на размеченных данных и реализовать алгоритм Витерби для задачи POS-теггинга (определение частей речи слов в тексте)

- научиться использовать ряд POS-теггеров из библиотеки NLTK и сравнить качество их работы

- придумать различные признаки для CRF и использовать их в реализации CRF из пакета CRFsuite для решения задачи NER (выделение именованных сущностей в тексте)

- использовать готовое решение для решения задачи NER и сравнить качество

#### Комментарии и советы:

1. Для выполнения потребуются модули Python numpy, nltk, pycrfsuite (для импорта последнего нужно установить пакет python-crfsuite).

2. Все необходимые для выполнения задания данные либо приложены, либо могут быть скачаны с помощью nltk.download().

3. Посмотреть параметры конструктора и других методов классов можно набрав и выполнив в ячейке с кодом '?full_method_name'.

4. В коде Stanford NER tagger, возможно, присутствует ошибка. Для её устранения в файле /usr/local/lib/python2.7/site-packages/nltk/tag/api.py (или его аналоге в Windows) замените строку с номером 66 на следующую: tagged_sents = self.tag_sents([untag(sent) for sent in gold])

### 1. Определение частей речи (POS)

Мы будем решать задачу определения частей речи (POS-теггинга) с помощью скрытой марковской модели (HMM). Формула совместной плотности наблюдаемых и скрытых переменных задается как

$$ p(X, T) = p(T) p(X|T) = p(t_1)  \prod_{i=2}^N p(t_i|t_{i-1}) \prod_{i=1}^N p(x_i|t_i)$$

В данном случае:

- наблюдаемые переменные $X$ - это слова корпуса;

- скрытые переменные $T$ - это POS-теги.

#### 1.1. Обучение HMM на размеченных данных

Требуется построить скрытую марковскую модель и настроить все ее параметры с помощью оценок максимального правдоподобия по размеченным данным (последовательности пар слово+тег):

- Вероятности переходов между скрытыми состояниями $p(t_i | t_{i - 1})$ посчитайте на основе частот биграмм POS-тегов.

- Вероятности эмиссий наблюдаемых состояний $p(x_i | t_i)$ посчитайте на основе частот "POS-тег - слово".

- Обратите внимание на проблему разреженности счетчиков и сделаейте все вероятности сглаженными по Лапласу (add-one smoothing).

- Распределение вероятностей начальных состояний $p(t_1)$ задайте равномерным.

Обратите внимание, что так как мы используем размеченные данные, то у нас нет необходимости в оценивании апостериорных вероятностей на скрытые переменные с помощью алгоритма forward-backword и использовании EM-алгоритма.

In [12]:
import numpy as np

In [57]:
class HMM:
    MAX_TAGS = 50
    MAX_WORDS = 50000
    
    def __init__(self, lower=True):
        self.lower = lower
    
    def fit(self, sentences):
        word_tag = np.ones((self.MAX_WORDS, self.MAX_TAGS))
        tag_tag = np.ones((self.MAX_TAGS, self.MAX_TAGS))
        words_dict = dict()
        tags_dict = dict()
        
        tag_i, word_i = 0, 0
        for sentence in sentences:
            prev_tag = None
            for word, tag in sentence:
                if self.lower:
                    word = word.lower()
                
                if word not in words_dict:
                    words_dict[word] = word_i
                    word_i += 1
                    
                if tag not in tags_dict:
                    tags_dict[tag] = tag_i
                    tag_i += 1
                    
                word_tag[words_dict[word], tags_dict[tag]] += 1
                if prev_tag is not None:
                    tag_tag[tags_dict[prev_tag], tags_dict[tag]] += 1
                    
                prev_tag = tag
        
        self.word_tag = np.copy(word_tag[:word_i, :tag_i])
        self.tag_tag = np.copy(tag_tag[:tag_i, :tag_i])
        self.words_dict = words_dict
        self.tags_dict = tags_dict
        
        del word_tag
        del tag_tag
        
        self.word_tag_prob = self.word_tag / self.word_tag.sum(axis=0)
        self.tag_tag_prob = self.tag_tag / self.tag_tag.sum(axis=1)[:, np.newaxis]
        self.tags_dist = np.ones(len(self.tags_dict)) / len(self.tags_dict)
        
    def predict(self, sentence):
        delta = np.zeros((len(self.tags_dict), len(sentence)))
        s = np.zeros((len(self.tags_dict), len(sentence) - 1), dtype=np.int)
        
        word = sentence[0].lower() if self.lower else sentence[0]
        delta[:, 0] = np.log(self.tags_dist) + np.log(self.word_tag_prob[self.words_dict[word]])
        tt_log = np.log(self.tag_tag_prob)
        for i, word in enumerate(sentence[1:]):
            if self.lower:
                word = word.lower()
                
            tmp = delta[:, i, np.newaxis] + tt_log +\
                  np.log(self.word_tag_prob[self.words_dict[word]])
                    
            s[:, i] = np.argmax(tmp, axis=0)
            delta[:, i + 1] = np.max(tmp, axis=0)
            
        tag_by_indx = sorted(self.tags_dict, key=self.tags_dict.get)
        
        tag_indx = np.argmax(delta[:, -1])
        prediction = [tag_by_indx[tag_indx]]
        for i in range(s.shape[1] - 1, -1, -1):
            tag_indx = s[tag_indx, i]
            prediction += [tag_by_indx[tag_indx]]
            
        return prediction[::-1]

Загрузите brown корпус с универсальной системой тегирования. Для этого вам понадобятся ресурсы brown и universal_tagset из nltk.download().  В этой системе содержатся следующие теги:

- ADJ - adjective (new, good, high, ...)
- ADP - adposition	(on, of, at, ...)
- ADV - adverb	(really, already, still, ...)
- CONJ	- conjunction	(and, or, but, ...)
- DET - determiner, article	(the, a, some, ...)
- NOUN	- noun	(year, home, costs, ...)
- NUM - numeral	(twenty-four, fourth, 1991, ...)
- PRT -	particle (at, on, out, ...)
- PRON - pronoun (he, their, her, ...)
- VERB - verb (is, say, told, ...)
- .	- punctuation marks	(. , ;)
- X	- other	(ersatz, esprit, dunno, ...)

Обратите внимание, что тегсеты в корпусах текстов и в различных теггерах могут быть разными. Проверять это можно, глядя на сами теги, а симптом - подозрительно низкое качество теггирования. В таких случаях рекомендуется всё приводить сперва к универсальному тегсету, а потом уже мерять качество. Полезной может оказаться эта ссылка http://www.nltk.org/_modules/nltk/tag/mapping.html

Проанализируйте данные, с которыми Вы работаете. В частности, ответьте на вопросы:
- Каков общий объем датасета, формат?
- Приведены ли слова к нижнему регистру? Чем  это нам может в дальнейшем помешать?
- Как распределены слова в корпусе?  Как распределены теги в корпусе? Подсчитайте частоты и отобразите любым удобным для Вас способом. Проинтерпретируйте полученные результаты.

Задем сделайте случайное разбиение выборки на обучение и контроль в отношении 9:1 и обучите скрытую марковскую модель из предыдущего пункта. Если впоследствии обучение моделей будет занимать слишком много времени, работайте с подвыборкой, например, только текстами определенных категорий.

In [ ]:
from nltk.corpus import brown

brown_tagged_sents = brown.tagged_sents(tagset="universal")

# you code here

In [8]:
from sklearn.model_selection import train_test_split

In [84]:
X_train, X_test = train_test_split(brown_tagged_set)

#### 1.2 Алгоритм Витерби для применения модели

Чтобы использовать обученную модель для определения частей речи на новых данных, необходимо реализовать алгоритм Витерби. Это алгоритм динамиеского программирования, с помощью которого мы будем находить наиболее вероятную последовательность скрытых состояний модели для фиксированной последовательности слов:

$$ \hat{T} = \arg \max_{T} p(T|X) = \arg \max_{T} p(X, T) $$

Определим функцию, определяющую максимальную вероятность последовательности, заканчивающейся на $i$-ой позиции в состоянии $k$:

$$\delta(k, i) = \max_{t_1, \dots t_{i-1}} p(x_1, \dots x_i, t_1, \dots t_i=k)$$

Тогда $\max_{k} \delta(k, N)$ - максимальная вероятность всей последовательности. А состояния, на которых эта вероятность достигается - ответ задачи.

Алгоритм Витерби заключается в последовательном пересчете функции $\delta(k, i)$ по формуле:

$$\delta(k, i) = \max_{m} \delta(m, i-1) p(t_i = k|t_{i-1} = m) p(x_i|t_i=k) $$

Аналогично пересчитывается функция, определяющая, на каком состоянии этот максимум достигается:

$$s(k, i) = \arg \max_{m} \delta(m, i-1) p(t_i = k|t_{i-1} = m) p(x_i|t_i=k) $$


На практике это означает заполнение двумерных массивов размерности: (длина последовательности) $\times$ (количество возможных состояний). Когда массивы заполнены, $\arg \max_{k} \delta(k, N)$ говорит о последнем состоянии. Начиная с него можно восстановить все состояния по массиву $s$. 

Осталось уточнить, как стартовать последовательный пересчет (чем заполнить первый столбец массива вероятностей):

$$\delta(k, 1) = p(k) p(x_1|t_1=k)$$

В реализации рекомендуется перейти к логарифмам, т.к. произведение большого числа маленьких вероятностей может приводить к вычислительным ошибкам.

Проверьте работу реализованного алгоритма на следующих модельных примерах, проинтерпретируйте результат.

- 'he can stay'
- 'a milk can'
- 'i saw a dog'
- 'an old saw'

In [88]:
hmm = HMM()

In [89]:
hmm.fit(X_train + X_test)

In [90]:
hmm.predict('he can stay'.split(' '))

[u'PRON', u'VERB', u'VERB']

In [91]:
hmm.predict('a milk can'.split(' '))

[u'DET', u'NOUN', u'VERB']

In [92]:
hmm.predict('i saw a dog'.split(' '))

[u'PRON', u'VERB', u'DET', u'NOUN']

In [93]:
hmm.predict('an old saw'.split(' '))

[u'DET', u'ADJ', u'VERB']

Примените модель к отложенной выборке Брауновского корпуса и подсчитайте точность определения тегов (accuracy). Сделайте выводы. 

In [ ]:
# your code here

#### 1.3. Готовые POS-теггеры из NLTK

В прошлом пункте Вы реализовали свой POS-тегер на основе скрытой марковской модели. Теперь сравните его работу с готовыми средставми, доступными в библиотеке NLTK: http://www.nltk.org/api/nltk.tag.html

Примерный набор кандидатов для сравнения:
- Простейший теггер, который всем словам ставит в соответствие одну и ту же метку
- Основанный на правилах RegexpTagger (правила можно поискать в Интернете или придумать самим)
- N-граммные теггеры (разберитесь и поэкспериментируйте с параметром backoff)
- Теггеры на основе графических моделей (можно взять только Stanford): 
    - HiddenMarkovModelTagger
    - CRFTagger
    - StanfordPOSTagger (потребуется .jar файл теггера и обученная модель (легко находятся в Интернете), чтобы подать на вход конструктору класса)
- BrillTagger, основанный на трансформациях

Если работа с какими-то модулями приводит к техническим проблемам, которые Вы не можете решить, это не страшно, модуль можно пропустить. Однако навык быстрого освоения документации / поиска моделей в гугле полезен.  Чем более полным и корректным будет сравнение, тем лучше.

При проведении экспериментов обращайте внимание на следующие моменты (и отразите их в отчете):
- Какой подход лежит в основе теггера
- На каких данных он обучен (если Вы скачали готовую модель)
- Сколько времени занимает обучение на brown корпусе (если обучаете сами)
- Какая точность получается на контролькой выборке (метод evaluate())

Сформируйте рекоммендиции о том, какую технологию Вы бы использовали, если встретитесь с задачей определения частей речи в будущем.

In [31]:
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import RegexpTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger
from nltk.tag import TrigramTagger
from nltk.tag import  HiddenMarkovModelTagger
from nltk.tag import CRFTagger
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tag import BrillTagger

# your code here

### 2. Выделение именованных сущностей (NER)



#### 2.1. Генерация признаков для CRF

Выделение именованных сущностей - другая распространенная задача разметки последовательности слов. Чаще всего она решается марковскими моделями максимальной энтропии (MEMM) или условными случайными полями (CRF). При этом основная сложность заключается в генерации  хороших признаков. 

В данном задании Вам требуется придумать и использовать множество признаков для обучения CRF из библиотеки CRFsuite. В этой библиотеке реализована linear-chain CRF с потенциалами двух типов (аналогично HMM):

$$ \psi_{mk}(t_{i-1}, t_{i}) = [t_{i-1} = m] \, [t_{i} = k]; \quad \psi_{jk}(t_{i}, x_i) = [t_{i} = k] \, f_j(x_i)$$


Потенциалы первого типа назвают transition features, они зависят только от биграмм меток. Потенциалы второго типа -- label-observation (node-observation) featrues; они зависят от метки и признаков наблюдаемого слова (observation features). Несмотря на то, что в формуле явно участвует текущее слово $x_i$, подход остается полностью корректным, когда признаки зависят также от контекста слова (соседних слов). Это следствие того, что CRF является дискриминативной моделью, и наблюдаемые переменные $X$ не моделируются. 


**Указания к заданию:** 
- Загрузите из NLTK обучающие и тестовые датасеты для задачи выделения именованных сущеностей CoNLL 2002 shared task на английском, испанском и голландском языках в BIO-нотации (nltk.corpus.conll2002).
- Для обучения CRF модели библиотеке необходимо передать последовательность наблюдаемых признаков $f_j(x_i)$ и меток $y_i$. Ниже приведен весь технический код, который позволит сконцентрироваться только на самом творческом этапе -- генерации признаков.
- Оцените качество приведенного решения. 
- Ваша задача заключается в том, чтобы повысить его. Помимо генерации новых признаков, можно обратить внимание на параметры обучения, в частности, feature.minfreq позволяет отсеивать редкие признаки.  
- При проверке задания будет оцениваться как достигнутое качество, так и разнообразие/оригинальность использованных признаков. Если вы попробовали какие-то признаки, но они не помогли, также включите их в отчет. 
- Если у Вас закончилась фантазия, почитайте обзоры и статьи по теме.


In [68]:
# Let's define very simple example features.

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word.isupper=%s' % word.isupper(),
        'postag=' + postag,
        # your code here
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            # your code here
        ])
    else:
        features.append('BOS')
        
    # your code here
    
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

In [133]:
# Let's prepare functions for more comfortable work with pycrfsuite.

import pycrfsuite

MODEL_NAME = 'model.crfsuite'

def train(train):
    X_train = [sent2features(s) for s in train]
    y_train = [sent2labels(s) for s in train]

    trainer = pycrfsuite.Trainer(verbose=False)

    trainer.set_params({'c1': 1.0, 'c2': 1e-3, 'max_iterations': 50,
                        'feature.possible_transitions': True})

    for xseq, yseq in zip(X_train, y_train):
        trainer.append(xseq, yseq)

    trainer.train(MODEL_NAME)

def evaluate(test):
    X_test = [sent2features(s) for s in test]
    y_test = [sent2labels(s) for s in test]

    tagger = pycrfsuite.Tagger()
    tagger.open(MODEL_NAME)

    y_pred = [tagger.tag(x) for x in X_test]

    true_counter, total_counter = 0.0, 0.0
    for p, t in zip(y_pred, y_test):
        assert len(p) == len(t)
        total_counter += len(p)
        true_counter += sum([str(i) == str(j) for i, j in zip(p, t)])
    return true_counter / total_counter

In [ ]:
# your code here

#### 2.2. Stanford NER tagger

Воспользуйтесь StanfordNERTagger для решения задачи NER на тех же тестовых данных, только для английского языка (обучать модель здесь не требуется). Приведите данные в соответствие нужному формату. Сравните результат с полученным выше. Настройка StanfordNERTagger производится аналогично настройке StanfordPOSTagger. В качестве готовой модели можно взять 'english.all.3class.distsim.crf.ser.gz'.

In [ ]:
# your code here